In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import re
import random
import argparse
from pathlib import Path
import json
import itertools
import time
from datetime import datetime
import shutil
import torch
import torch.nn.functional as F
import numpy as np
import math
import cv2
from torchvision import transforms
from PIL import Image
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from diffusers import AutoencoderKL, DDPMScheduler, UNet2DConditionModel, ControlNetModel, StableDiffusionControlNetPipeline, DDIMScheduler


from ip_adapter.utils import is_torch2_available

import torch.utils.checkpoint
import matplotlib.pyplot as plt

from pipe_infer import StableDiffusionInstantIDPipeline


In [ ]:
pretrained_model_name_or_path ="models/Realistic_Vision_V4.0_noVAE"

ip_ckpt = "{checkpoint}" 

adapter = ip_ckpt + "/pytorch_model.bin"
controlnet_path =  ip_ckpt + "/controlnet"


# Load pipeline
controlnet = ControlNetModel.from_pretrained(controlnet_path, torch_dtype=torch.float16)

base_model_path =pretrained_model_name_or_path
    
pipe =  StableDiffusionInstantIDPipeline.from_pretrained(
        base_model_path,
        controlnet=controlnet,
        torch_dtype=torch.float16,
    )

pipe.cuda()
pipe.load_ip_adapter_instantid(adapter)

In [ ]:

contour_image = Image.open("{contour_image_path}")

ori_wide,ori_heigh = contour_image.size
contour_image = contour_image.resize((256,256))


In [2338]:
import random
import cv2

def random_crop_with_outline(image_path, outline_path, output_path):
    # 读取图像和轮廓图
    image = cv2.imread(image_path)
    outline = cv2.imread(outline_path)

    # 将轮廓图转换为灰度图
    gray_outline = cv2.cvtColor(outline, cv2.COLOR_BGR2GRAY)

    # 应用二值化以获取轮廓区域
    _, binary_outline = cv2.threshold(gray_outline, 240, 255, cv2.THRESH_BINARY_INV)

    # 找到轮廓
    contours, _ = cv2.findContours(binary_outline, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 随机裁剪
    crop_size = 100

    # 找到所有轮廓的边界框
    bboxes = [cv2.boundingRect(contour) for contour in contours]

    valid_crops = []

    for (x, y, w, h) in bboxes:
            # 计算可以裁剪的区域，确保裁剪区域在轮廓内
        for i in range(max(0, x), min(x + w - crop_size + 1, image.shape[1] - crop_size + 1)):
            for j in range(max(0, y), min(y + h - crop_size + 1, image.shape[0] - crop_size + 1)):
                valid_crops.append((i, j))


    if valid_crops:
        # 随机选择一个有效的裁剪位置
        idx = random.randint(0, len(valid_crops) - 1)
        x, y = valid_crops[idx]

        # 裁剪图像
        cropped_image = image[y:y + crop_size, x:x + crop_size]
        cv2.imwrite(output_path, cropped_image)
        print(f'Cropped image saved: {output_path}')
    else:
        # print("No valid crop found within the contour.")
        center_x = image.shape[1] // 2
        center_y = image.shape[0] // 2

            # 计算裁剪区域的起始点
        x = max(center_x - crop_size // 2, 0)
        y = max(center_y - crop_size // 2, 0)

            # 确保裁剪区域不超出图像边界
        x = min(x, image.shape[1] - crop_size)
        y = min(y, image.shape[0] - crop_size)

            # 裁剪图像
        cropped_image = image[y:y + crop_size, x:x + crop_size]
        cv2.imwrite(output_path, cropped_image)
     


In [ ]:
#randomcrop

outline_path = "{contour_image_path}"
image_path = "{image_path}"
# out = "pamtrain_lossft_crop_out/top_crop_2.png"
out = "crop.png"


# out = "crop.png"

random_crop_with_outline(image_path, outline_path, out)



In [ ]:
#get colors  38、63
import numpy as np
import cv2
from sklearn.cluster import KMeans

# 读取图像
#img = cv2.imread("/data/rensisi/InstantID/style/17.jpg")
img = cv2.imread("{image_path}")

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 将图像转换为2D数组
pixels = img.reshape(-1, 3)

# 使用K-means聚类算法提取前5种颜色
kmeans = KMeans(n_clusters=5, random_state=0).fit(pixels)
colors = kmeans.cluster_centers_.astype(np.uint8)

# 创建新的图像
size = 128
new_img = np.zeros((size, size, 3), dtype=np.uint8)

# 在新图像上绘制色条
for i in range(5):
    new_img[:, i*(size//5):(i+1)*(size//5)] = colors[i]
img = new_img
plt.imshow(new_img)
plt.axis('off')  # 隐藏轴
plt.show()
# 保存新的图像
# 保存新的图像
cv2.imwrite("color.png", cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR))

text+colors

In [ ]:

seed = random.randint(100, 2**10)
# seed = 415
print(seed)

prompt = "French floral chiffon dress,no background,high quality"  
  
n_prompt = "(lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured (lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch,deformed, mutated, cross-eyed, ugly, disfigured"

generator = torch.Generator(device="cpu").manual_seed(seed)  

pipe.set_ip_adapter_scale(0.8) 
image =  pipe(
            prompt=prompt,
            negative_prompt=n_prompt,
            image_embeds=img,
            image=contour_image,
            controlnet_conditioning_scale=0.0,
            num_inference_steps=30,
            guidance_scale=8, 
            generator=generator,
            ).images[0]

image 

sketch+crop

In [ ]:

seed = random.randint(400, 2**10)
seed= 1020

print("seed:" , seed)
prompt = "Casual style jacket" 
n_prompt = "(lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured (lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch,deformed, mutated, cross-eyed, ugly, disfigured"

generator = torch.Generator(device="cpu").manual_seed(seed) 
#
pipe.set_ip_adapter_scale(0.8) 
image =  pipe(
            prompt=prompt,
            negative_prompt=n_prompt,
            image_embeds=img,
            image=contour_image,
            controlnet_conditioning_scale=0.9,
            num_inference_steps=30,
            guidance_scale=5, 
            generator=generator,
            ).images[0]

image

text + crop + contour

In [ ]:

seed = random.randint(100, 2**10)
seed = 869
print(seed)
prompt = "Tie-dye, bohemian style t-shirt,high quality."  #
n_prompt = "(lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured (lowres, low quality, worst quality:1.2), (text:1.2), watermark, painting, drawing, illustration, glitch,deformed, mutated, cross-eyed, ugly, disfigured"

generator = torch.Generator(device="cpu").manual_seed(seed)  

pipe.set_ip_adapter_scale(0.8)
image =  pipe(
            prompt=prompt,
            negative_prompt=n_prompt,
            image_embeds=img,
            image=contour_image,
            controlnet_conditioning_scale=0.7,
            num_inference_steps=30,
            guidance_scale=6, 
            generator=generator,
            ).images[0]
#all train


image